In [ ]:
import pandas as pd
import os
from pathlib import Path
import func_proc_filepath as mFILE
import func_proc_image as mIMAGE
import cv2

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def main():
    # root folder
    dir_root = str(Path(Path().resolve()).parent)
    # input: folder path including videos
    dir_img = dir_root + "\\src\\videos"
    # input: file path of vioedo list
    file_list = dir_root + "\\parameters\\file_pos_list.csv"

    # output: folder of output
    dir_out = dir_root + "\\dataset\\aff2_images\\cropped"
    if os.path.isdir(dir_out) == False:
        os.makedirs(dir_out)

    # load list
    df_files = pd.read_csv(file_list)
    #print(df_files[50:100])

    num_files = len(df_files)
    #num_files=10
    
    for NUM in range(num_files):        
        # ****
        
        # get params
        fp = dir_img + "\\" + df_files["file"][NUM]
        main_f, main_x, main_y = df_files["main"][NUM], df_files["main_x"][NUM], df_files["main_y"][NUM]
        left_f, left_x, left_y = df_files["left"][NUM], df_files["left_x"][NUM], df_files["left_y"][NUM]
        right_f, right_x, right_y = df_files["right"][NUM], df_files["right_x"][NUM], df_files["right_y"][NUM]
        # get filename without ext
        name = os.path.splitext(os.path.basename(fp))[0]

        print(main_f, left_f, right_f)

        if main_f == 1:
            dir_out_main = dir_out + "\\" + name
            if os.path.isdir(dir_out_main) == False:
                os.makedirs(dir_out_main)
        if left_f == 1:
            dir_out_left = dir_out + "\\" + name + "_left"
            if os.path.isdir(dir_out_left) == False:
                os.makedirs(dir_out_left)
        if right_f == 1:
            dir_out_right = dir_out + "\\" + name + "_right"
            if os.path.isdir(dir_out_right) == False:
                os.makedirs(dir_out_right)

        # read movies
        cap = cv2.VideoCapture(fp)
        if not cap.isOpened():
            print("fili not open")

        cnt=0
        IMG_SIZE=300
        TH_CONF=0.9
        #max_cnt=len(max_cnt)

        all_frame = round(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        #all_frame = 30

        # repeat all frame
        #while(cap.isOpened()):
        for i in range(all_frame):
            # get frame
            ret, frame = cap.read()

            # フレームを表示
            #mIMAGE.display_cv_image(frame, format='.jpg')
            #cv2.imshow("Frame", frame)
            if ret == True:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                #pad = 0.1
                res_list = mIMAGE.split_image_per_face(frame, base_size=IMG_SIZE, th_conf=TH_CONF)
                length = len(res_list)
                if length<1:
                    cnt=cnt+1
                    continue
                else:
                    best_m, best_l, best_r = 999999999, 999999999, 999999999
                    img_m, img_l, img_r = 0, 0, 0
                    for l in range(length):
                        imx = res_list[l][1]
                        imy = res_list[l][2]
                        dis_m = (main_x-imx)**(2) + (main_y-imy)**(2)
                        dis_l = (left_x-imx)**(2) + (left_y-imy)**(2)
                        dis_r = (right_x-imx)**(2) + (right_y-imy)**(2)
                        if best_m > dis_m:
                            best_m = dis_m
                            img_m = l
                        if best_l > dis_l:
                            best_l = dis_l
                            img_l = l
                        if best_r > dis_r:
                            best_r = dis_r
                            img_r = l
                    best_img_m = res_list[img_m][0]
                    best_img_l = res_list[img_l][0]
                    best_img_r = res_list[img_r][0]

                    if main_f == 1:
                        file_out = dir_out_main + "\\{0:000006d}.jpg".format(cnt)
                        cv2.imwrite(file_out, best_img_m)
                    if left_f == 1:
                        file_out = dir_out_left + "\\{0:000006d}.jpg".format(cnt)
                        cv2.imwrite(file_out, best_img_l)
                    if right_f == 1:
                        file_out = dir_out_right + "\\{0:000006d}.jpg".format(cnt)
                        cv2.imwrite(file_out, best_img_r)

                    cnt=cnt+1
            else:
                cnt=cnt+1

        cap.release()
        cv2.destroyAllWindows()
        
        log = "{0}/{1} is finished".format(NUM, num_files-1)
        print(log)
        

In [ ]:
if __name__ == "__main__":
    main()